In [1]:
just_checking_integrity=False

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
items_features = pd.read_csv('../dataset/processed_data/simplified_features_and_categories_30.csv')
items_features.head()

,item_id,feature_idx
0,19021,0
1,19021,22
2,19021,15
3,19021,446
4,19021,1


In [4]:
items_features['item_id'].max()

23691

In [5]:
dataset = np.zeros(
    (items_features['item_id'].max()+1, items_features['feature_idx'].max()+1)
    )
dataset[(items_features['item_id'], items_features['feature_idx'])] = 1

dataset
train_dataset=dataset[1:,:]

In [6]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(train_dataset,test_size=0.2,random_state=941116)

In [7]:
dataset.shape

(23692, 636)

In [8]:
train.shape

(18952, 636)

In [9]:
test.shape

(4739, 636)

# Autoencoder

In [10]:
latent_dimension = 64
starting_dimension = 512

inputs = layers.Input(int(items_features['feature_idx'].max()+1))

x = inputs 
dimension = starting_dimension

while(dimension != latent_dimension):
    #x = layers.BatchNormalization()(x)
    x = layers.Dense(units=dimension, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
    x = layers.Dropout(0.1)(x)

    dimension = dimension//2

x = layers.Dense(units=dimension,activation="tanh",activity_regularizer=tf.keras.regularizers.L1L2(l1=5e-5,l2=4e-4))(x)
outputs = x
encoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='encoder')

In [11]:
max_width = 1024
ending_dimension = items_features['feature_idx'].max()+1

inputs = layers.Input(latent_dimension)

x = inputs 
dimension = latent_dimension

x = layers.Dense(units=256, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(units=512, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.2)(x)
    

x = layers.Dense(units=ending_dimension,activation="sigmoid")(x)
outputs = x

decoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='decoder')

#decoder.summary()

In [12]:
ae = tf.keras.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs), name='auto_encoder')
ae.summary()

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 636)]             0         
                                                                 
 dense (Dense)               (None, 512)               326144    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)              

In [13]:
def custom_loss(x_true,x_pred):
    return tf.keras.losses.cosine_similarity(x_true,x_pred)+tf.keras.losses.mean_squared_error(x_true,x_pred)#

In [14]:
ae.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.006), 
    loss=custom_loss,
    metrics=[tf.keras.metrics.CosineSimilarity(),tf.keras.metrics.MeanAbsoluteError()]
    )

# Training

In [15]:
ae.fit(
    train,
    train, 
    batch_size=64,
    validation_data=(test,test),
    epochs=5 if just_checking_integrity else 200,
    shuffle=True,
    callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='cosine_similarity',
                patience=10,
                restore_best_weights=True,
                mode="max"
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_cosine_similarity',
                factor=0.7,
                patience=2,
                min_lr=5e-5,
                mode="max",
                min_delta=0.0001,
            )
        ]
    )

Epoch 1/200
297/297 [==============================] - 3s 7ms/step - loss: -0.7552 - cosine_similarity: 0.7932 - mean_absolute_error: 0.0499 - val_loss: -0.8418 - val_cosine_similarity: 0.8666 - val_mean_absolute_error: 0.0319 - lr: 0.0060
Epoch 2/200
297/297 [==============================] - 2s 7ms/step - loss: -0.8501 - cosine_similarity: 0.8735 - mean_absolute_error: 0.0307 - val_loss: -0.8715 - val_cosine_similarity: 0.8922 - val_mean_absolute_error: 0.0272 - lr: 0.0060
Epoch 3/200
297/297 [==============================] - 2s 7ms/step - loss: -0.8705 - cosine_similarity: 0.8912 - mean_absolute_error: 0.0267 - val_loss: -0.8875 - val_cosine_similarity: 0.9063 - val_mean_absolute_error: 0.0235 - lr: 0.0060
Epoch 4/200
297/297 [==============================] - 2s 7ms/step - loss: -0.8835 - cosine_similarity: 0.9025 - mean_absolute_error: 0.0241 - val_loss: -0.9000 - val_cosine_similarity: 0.9169 - val_mean_absolute_error: 0.0215 - lr: 0.0060
Epoch 5/200
297/297 [===================

In [16]:
temp=encoder(dataset)

In [17]:
temp.shape

TensorShape([23692, 64])

In [18]:
temp

<tf.Tensor: shape=(23692, 64), dtype=float32, numpy=
array([[-0.01806994,  0.0089377 ,  0.15051028, ...,  0.03708984,
         0.01504658, -0.0039938 ],
       [ 0.04988107, -0.02240731, -0.17651683, ...,  0.04909024,
        -0.0692801 , -0.08338314],
       [ 0.04035887, -0.03615299, -0.09513128, ..., -0.04654828,
        -0.09808538, -0.04732575],
       ...,
       [ 0.14372921,  0.06672861, -0.14092936, ..., -0.1015987 ,
        -0.02589842, -0.00743572],
       [ 0.03895115,  0.03133077, -0.03150491, ...,  0.16357984,
         0.29659215,  0.25815088],
       [ 0.04904211,  0.03686346, -0.2726057 , ...,  0.0152264 ,
         0.3298344 ,  0.02656483]], dtype=float32)>

In [19]:
tf.reduce_sum(temp,axis=1).numpy().max(),tf.reduce_sum(temp,axis=1).numpy().min(),tf.reduce_sum(temp,axis=1).numpy().std()

(3.9407606, -3.9119434, 0.94978845)

In [20]:
np.save('../dataset/processed_data/compressed_features', encoder(dataset))